In [1073]:
import numpy as np
import pandas as pd
import os

In [1074]:
os.chdir(r"C:\Users\sreek\Documents\1. Praxis\3. Term 3\3. CAPP- New")

In [1075]:
data = pd.read_csv("Master data - Sample 2 Non Financial_ copy.csv")

In [1076]:
data.columns

Index(['Org_ID', 'Date_published', 'Enc_ID', 'Headline', 'Synopsis',
       'Headline + Synop', 'Full_text', 'A-1', 'A-1 Status', 'A-11',
       'A-11 Status', 'K-33', 'K-33 Status', 'Mean rating', 'Final Status',
       'Match b/w team - Manual', 'Vader_compound_score', 'Vader- Status',
       'lm_polarity', 'LM- Status', 'pattern_polarity', 'Pattern - Status',
       'Sample Vs Vader', 'Sample Vs LM', 'Sample Vs Pattern'],
      dtype='object')

In [1077]:
# Defining test size to try with different test sizes
testsize = 0.2

In [1078]:
nfd = data[["Headline + Synop","Full_text","Final Status"]]

In [1079]:
nfd.head(2)

,Headline + Synop,Full_text,Final Status
0,India Post Payments Bank introduces issuer cha...,Getty ImagesThe India Post Payments Bank (IPPB...,Positive
1,RBI directs banks to offer cardless cash withd...,Getty ImagesThe Reserve Bank of India (RBI) ha...,Positive


In [1080]:
nfd['Final Status'].value_counts()

Positive    241
Negative    159
Name: Final Status, dtype: int64

In [1081]:
# Label the final status 
nfd['Final Status']=nfd['Final Status'].apply(lambda x: 1 if  x=="Positive" else 0)
nfd.head()

C:\Users\sreek\AppData\Local\Temp/ipykernel_5440/1311584812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfd['Final Status']=nfd['Final Status'].apply(lambda x: 1 if  x=="Positive" else 0)


,Headline + Synop,Full_text,Final Status
0,India Post Payments Bank introduces issuer cha...,Getty ImagesThe India Post Payments Bank (IPPB...,1
1,RBI directs banks to offer cardless cash withd...,Getty ImagesThe Reserve Bank of India (RBI) ha...,1
2,Mahindra to explore more partnerships for EV p...,ReutersDavos: India's Mahindra and Mahindra wi...,1
3,Elon Musk bags $7.1 billion in new financing f...,Elon Musk has secured about $7.1 billion of ne...,1
4,"Hexagon freezes Russian business, books charge...",ETtechStockholm: Measurement technology group ...,0


In [1082]:
nfd.shape

(400, 3)

In [1083]:
nfd['Final Status'].value_counts()

1    241
0    159
Name: Final Status, dtype: int64

In [1084]:
# Pre processing : 

In [1085]:
# Importing stopwords from nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [1086]:
# Removing some words from the stopwords list as they contain a meaning for our problem statement 
for i in ['further','over','under','up','down','above','below']:
  stopwords.remove(i)

In [1087]:
# An empty column for the preprocessed and marked text which is ready for embedding
nfd['text_processed']=''

C:\Users\sreek\AppData\Local\Temp/ipykernel_5440/976053868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfd['text_processed']=''


In [1088]:
articles=[]
for i in range(0,nfd.shape[0]):
    articles.append(nfd.iloc[i])

## Headline syn work 

In [1089]:
# An empty column for the preprocessed and marked text which is ready for embedding
nfd['headsyntext_processed']=''

C:\Users\sreek\AppData\Local\Temp/ipykernel_5440/1917234557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfd['headsyntext_processed']=''


In [1090]:
articles=[]
for i in range(0,nfd.shape[0]):
    articles.append(nfd.iloc[i])

In [1091]:
# Preprocessing

import re
import string
def preprocess(article):
  words = [word for word in str(article['Headline + Synop']).split() if word not in stopwords] # List Comprehension
  text = " ".join(words)
  #Remove all the punctuation from the text
  text = re.sub(r'[^\w\s]', '', text)
  #Removes all numericals leaving the alphabets
  text = ''.join([i for i in text if not i.isdigit()])
  # Removing multiple spaces in the text and replacing them with a single space
  text = re.sub(' +', ' ', text)
  text = text.lower()
  article.headsyntext_processed = text

  return ("Cleaning Done!")


In [1092]:
import time
from timeit import default_timer as timer
from joblib import Parallel, delayed

start=timer()
res = Parallel(
    backend='threading',
    n_jobs=-1
)(delayed(preprocess)(x) for x in articles)
print('Done')
end=timer()
print(((end-start)/60),"mins")

C:\Users\sreek\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


Done
0.0018144633333273912 mins


In [1093]:
df = pd.DataFrame(articles)
df.head()

,Headline + Synop,Full_text,Final Status,text_processed,headsyntext_processed
0,India Post Payments Bank introduces issuer cha...,Getty ImagesThe India Post Payments Bank (IPPB...,1,,india post payments bank introduces issuer cha...
1,RBI directs banks to offer cardless cash withd...,Getty ImagesThe Reserve Bank of India (RBI) ha...,1,,rbi directs banks offer cardless cash withdraw...
2,Mahindra to explore more partnerships for EV p...,ReutersDavos: India's Mahindra and Mahindra wi...,1,,mahindra explore partnerships ev parts ceo ani...
3,Elon Musk bags $7.1 billion in new financing f...,Elon Musk has secured about $7.1 billion of ne...,1,,elon musk bags billion new financing twitter o...
4,"Hexagon freezes Russian business, books charge...",ETtechStockholm: Measurement technology group ...,0,,hexagon freezes russian business books charge ...


In [1094]:
from sklearn.model_selection import train_test_split

X = df['headsyntext_processed']
y =df['Final Status']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=testsize, random_state=42)

In [1095]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
# Naïve Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])



#Decision Tree 
text_clf_DT = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', DecisionTreeClassifier(max_depth = 3)),
])

In [1096]:
text_clf_nb.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [1097]:
# Form a prediction set
predictions = text_clf_nb.predict(X_test)

In [1098]:
# Print a classification report
print("Naive Bayes:")
print("\n")
print(metrics.classification_report(y_test,predictions))

Naive Bayes:


              precision    recall  f1-score   support

           0       0.67      0.05      0.10        39
           1       0.52      0.98      0.68        41

    accuracy                           0.53        80
   macro avg       0.59      0.51      0.39        80
weighted avg       0.59      0.53      0.39        80



In [1099]:
text_clf_DT.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', DecisionTreeClassifier(max_depth=3))])

In [1100]:
text_clf_DT.score(X_train, y_train)

0.684375

In [1101]:
# Form a prediction set
predictions = text_clf_DT.predict(X_test)

In [1102]:
# Print a classification report
print("Decision Tree:")
print("\n")
print(metrics.classification_report(y_test,predictions))

Decision Tree:


              precision    recall  f1-score   support

           0       0.67      0.36      0.47        39
           1       0.58      0.83      0.68        41

    accuracy                           0.60        80
   macro avg       0.62      0.59      0.57        80
weighted avg       0.62      0.60      0.58        80



In [1103]:
# importing random forest classifier from assemble module
from sklearn.ensemble import RandomForestClassifier

In [1104]:
text_clf_rf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier(max_depth = 5)),
])


In [1105]:
text_clf_rf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier(max_depth=5))])

In [1106]:
text_clf_rf.score(X_train, y_train)

0.63125

In [1107]:
text_clf_rf.score(X_test, y_test)

0.5125

## Full Text continued

In [1108]:
# Preprocessing

import re
import string
def preprocess(article):
  words = [word for word in str(article['Full_text']).split() if word not in stopwords] # List Comprehension
  text = " ".join(words)
  #Remove all the punctuation from the text
  text = re.sub(r'[^\w\s]', '', text)
  #Removes all numericals leaving the alphabets
  text = ''.join([i for i in text if not i.isdigit()])
  # Removing multiple spaces in the text and replacing them with a single space
  text = re.sub(' +', ' ', text)
  text = text.lower()
  article.text_processed = text

  return ("Cleaning Done!")


In [1109]:
import time
from timeit import default_timer as timer
from joblib import Parallel, delayed

start=timer()
res = Parallel(
    backend='threading',
    n_jobs=-1
)(delayed(preprocess)(x) for x in articles)
print('Done')
end=timer()
print(((end-start)/60),"mins")

C:\Users\sreek\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


Done
0.008216404999984661 mins


In [1110]:
df = pd.DataFrame(articles)
df.head()

,Headline + Synop,Full_text,Final Status,text_processed,headsyntext_processed
0,India Post Payments Bank introduces issuer cha...,Getty ImagesThe India Post Payments Bank (IPPB...,1,getty imagesthe india post payments bank ippb ...,india post payments bank introduces issuer cha...
1,RBI directs banks to offer cardless cash withd...,Getty ImagesThe Reserve Bank of India (RBI) ha...,1,getty imagesthe reserve bank india rbi asked b...,rbi directs banks offer cardless cash withdraw...
2,Mahindra to explore more partnerships for EV p...,ReutersDavos: India's Mahindra and Mahindra wi...,1,reutersdavos indias mahindra mahindra explore ...,mahindra explore partnerships ev parts ceo ani...
3,Elon Musk bags $7.1 billion in new financing f...,Elon Musk has secured about $7.1 billion of ne...,1,elon musk secured billion new financing commit...,elon musk bags billion new financing twitter o...
4,"Hexagon freezes Russian business, books charge...",ETtechStockholm: Measurement technology group ...,0,ettechstockholm measurement technology group h...,hexagon freezes russian business books charge ...


In [1111]:
df.head()

,Headline + Synop,Full_text,Final Status,text_processed,headsyntext_processed
0,India Post Payments Bank introduces issuer cha...,Getty ImagesThe India Post Payments Bank (IPPB...,1,getty imagesthe india post payments bank ippb ...,india post payments bank introduces issuer cha...
1,RBI directs banks to offer cardless cash withd...,Getty ImagesThe Reserve Bank of India (RBI) ha...,1,getty imagesthe reserve bank india rbi asked b...,rbi directs banks offer cardless cash withdraw...
2,Mahindra to explore more partnerships for EV p...,ReutersDavos: India's Mahindra and Mahindra wi...,1,reutersdavos indias mahindra mahindra explore ...,mahindra explore partnerships ev parts ceo ani...
3,Elon Musk bags $7.1 billion in new financing f...,Elon Musk has secured about $7.1 billion of ne...,1,elon musk secured billion new financing commit...,elon musk bags billion new financing twitter o...
4,"Hexagon freezes Russian business, books charge...",ETtechStockholm: Measurement technology group ...,0,ettechstockholm measurement technology group h...,hexagon freezes russian business books charge ...


In [1112]:
nfd = df.drop("Full_text",axis = 1)

In [1113]:
from sklearn.model_selection import train_test_split

X = nfd['text_processed']
y = nfd['Final Status']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=testsize, random_state=42)

In [1114]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# Naïve Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])



#Decision Tree 
text_clf_DT = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', DecisionTreeClassifier(max_depth = 3)),
])

In [1115]:
text_clf_nb.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [1116]:
# Form a prediction set
predictions = text_clf_nb.predict(X_test)

In [1117]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 1 38]
 [ 0 41]]


In [1118]:
# Print a classification report
print("Naive Bayes:")
print("\n")
print(metrics.classification_report(y_test,predictions))

Naive Bayes:


              precision    recall  f1-score   support

           0       1.00      0.03      0.05        39
           1       0.52      1.00      0.68        41

    accuracy                           0.53        80
   macro avg       0.76      0.51      0.37        80
weighted avg       0.75      0.53      0.37        80



In [1119]:
text_clf_DT.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', DecisionTreeClassifier(max_depth=3))])

In [1120]:
text_clf_DT.score(X_train, y_train)

0.76875

In [1121]:
text_clf_DT.score(X_test, y_test)

0.675

In [1122]:
# Form a prediction set
predictions = text_clf_DT.predict(X_test)

In [1123]:
# Print a classification report
print("Decision Tree:")
print("\n")
print(metrics.classification_report(y_test,predictions))

Decision Tree:


              precision    recall  f1-score   support

           0       1.00      0.33      0.50        39
           1       0.61      1.00      0.76        41

    accuracy                           0.68        80
   macro avg       0.81      0.67      0.63        80
weighted avg       0.80      0.68      0.63        80



In [1124]:
text_clf_rf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier(max_depth = 5)),
])

In [1125]:
text_clf_rf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier(max_depth=5))])

In [1126]:
text_clf_rf.score(X_train, y_train)

0.7375

In [1127]:
text_clf_rf.score(X_test, y_test)

0.5375